In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [2]:
# FOLDER
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

In [3]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY = json.load(file)

CCL = pyccl.Cosmology(
    h = COSMOLOGY['H'],
    w0 = COSMOLOGY['W0'],
    wa = COSMOLOGY['WA'], 
    n_s = COSMOLOGY['NS'], 
    A_s = COSMOLOGY['AS'],
    m_nu = COSMOLOGY['M_NU'],  
    Neff = COSMOLOGY['N_EFF'],
    Omega_b = COSMOLOGY['OMEGA_B'], 
    Omega_k = COSMOLOGY['OMEGA_K'], 
    Omega_c = COSMOLOGY['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# Comparison
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=CCL, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=CCL, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

In [5]:
# Numerical integral

def integral_I7(chi1, chi2, chi3, chi4, chi5, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        result = (chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1)
        
        result = result * (- chi + (chi5 - chi4) / 2 + chi * chi4 * numpy.log(chi5 / chi4) / (chi5 - chi4))
        
        result = result * (((chi / chi2) ** 3) * power2)  
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    def formula_n(chi):
        result = (chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1)
        
        result = result * (- chi + (chi5 - chi4) / 2 + chi * chi4 * numpy.log(chi5 / chi4) / (chi5 - chi4))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

In [ ]:
# Coefficient

def coefficient_J7(chi1, chi2, chi3, chi4, chi5, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    c = (chi5 - chi4) / (2 * chi2)
    d = chi4 * numpy.log(chi5 / chi4) / (chi5 - chi4) - 1
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (1 / (5040 * b)) * (b * (2 * d * (432 + z * ( - 129 + 17 * z) + 12 * b * (21 + ( - 7 + z) * z)) + 3 * c * (350 + z * ( - 124 + 19 * z) + 14 * b * (15 + ( - 6 + z) * z))) - 
6 * (1 + b) * (5 * d * (28 + ( - 8 + z) * z) + 8 * c * (21 + ( - 7 + z) * z)) * numpy.log(1 + b))
    else: 
        formula = (1 / (25200 * a ** 4 * b)) * (a * b * ( - 420 * (3 * c + d) * p * z ** 2 + 70 * a ** 2 * ( - 30 * (2 * c + d) * p - 6 * (5 * c * (4 + p) + 2 * d * (5 + p)) * z + (d * ( - 6 + p) + 3 * c * ( - 5 + p)) * z ** 2) + 
210 * a * z * (8 * d * p + 10 * c * z + d * (4 + p) * z + c * p * (20 + 3 * z)) + a ** 6 * d * ( - 5950 + 4095 * p + 42 * (195 - 74 * z) * z + 4 * p * z * ( - 1554 + 625 * z)) + 
35 * a ** 3 * (c * (60 * (6 + p) - 20 * ( - 6 + p) * z + ( - 10 + 3 * p) * z ** 2) + d * (240 - 4 * ( - 15 + z) * z + p * (30 + ( - 8 + z) * z))) + 
7 * a ** 5 * (c * (1500 - 850 * p + 6 * p * (195 - 74 * z) * z + 5 * z * ( - 340 + 117 * z)) + d * ( - 25 * ( - 64 + 35 * p + 70 * z) + 2 * z * (p * (594 - 224 * z) + 297 * z) + 
30 * b * ( - 30 + 20 * p + 5 * (8 - 3 * z) * z + 6 * p * z * ( - 5 + 2 * z)))) + 
7 * a ** 4 * (d * (50 * ( - 12 + p) - 20 * ( - 5 + p) * z + ( - 10 + 3 * p) * z ** 2 - 150 * b * ( - 4 * (3 + ( - 3 + z) * z) + p * (6 + z * ( - 8 + 3 * z)))) + 
c * (100 * ( - 9 + p + 2 * z) + z * ( - 25 * z + p * ( - 50 + 9 * z)) - 150 * b * ( - 4 * (3 + ( - 3 + z) * z) + p * (6 + z * ( - 8 + 3 * z)))))) - 
420 * (( - 1 + a) ** 3 * b * (5 * a ** 2 * (6 * a * c - 4 * ( - 1 + a) * a * d - ((2 + 4 * a) * c + d + (2 - 3 * a) * a * d) * p) + 
2 * a * ( - 10 * a * (1 + 2 * a) * c + 5 * ( - 1 + a) * a * (1 + 3 * a) * d + 5 * (1 + a * (2 + 3 * a)) * c * p - 2 * ( - 1 + a) * (1 + 3 * a + 6 * a ** 2) * d * p) * z + 
(5 * a * (1 + a * (2 + 3 * a)) * c - 2 * ( - 1 + a) * a * (1 + 3 * a + 6 * a ** 2) * d - (3 * (1 + a * (2 + a * (3 + 4 * a))) * c + d + a * (2 + a * (3 + 2 * (2 - 5 * a) * a)) * d) * p) * z ** 2) * numpy.log(1 - a) + a ** 5 * (1 + b) * (d * (20 * (3 + ( - 3 + a) * a) - 5 * (6 + a * ( - 8 + 3 * a)) * p + 2 * ( - 30 + 5 * (8 - 3 * a) * a + 20 * p + 6 * a * ( - 5 + 2 * a) * p) * z + 
(20 - 15 * p + 2 * a * ( - 15 + 6 * a + 12 * p - 5 * a * p)) * z ** 2) + c * (20 * (3 + ( - 3 + z) * z) + a * ( - 30 + 20 * p + 5 * (8 - 3 * z) * z + 6 * p * z * ( - 5 + 2 * z)) - 
5 * p * (6 + z * ( - 8 + 3 * z)))) * numpy.log(1 + b)))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])
CHI3 = numpy.float128(CHI_GRID[GRID_INDEX + 2])
CHI4 = numpy.float128(CHI_GRID[GRID_SIZE - 1])
CHI5 = numpy.float128(CHI_GRID[GRID_SIZE])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 4957.1305889213954288 6946.038887321783477 6954.2547327845313703 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

50192.51002600693547 50192.510017882223462 -1.6187100433922435583e-10


In [9]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]
CHI4 = CHI_GRID[GRID_SIZE - 1]
CHI5 = CHI_GRID[GRID_SIZE]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 88.58943766284152 6946.0388873217835 6954.254732784531 0.0 1826.8838600584127 0.0 0.01


In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J7(CHI1, CHI2, CHI3, CHI4, CHI5, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

833200.5686217467 833200.568621745 -2.1094237467877974e-15
